In [1]:
include("JQlaw.jl")
using .JQlaw
using Plotly

WebIO._IJuliaInit()

## Simple Orbit Raise Example

In [3]:
orbit_inital = KeplarianOrbit(
                450000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

orbit_target = KeplarianOrbit(
                600000. + R_e,
                0.001,
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0.001),
                deg2rad(0)
                )

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.5,
                0,
                10, 
                600.,
                10^4,
                10^3)

Sat_Params = sat_params(100, 0.8, 0.5/100., 3100)

time_initial = 59715.0 # MJD

Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial)

print("Spent Prop: ", mass_hist[1] - mass_hist[end], "[kg]\n")
print("Transfer Time: ", time_hist[end] - time_hist[1], "[days]\n")

# change effectivity ratio to be more propellant safe...

Q_Params = QParams(
                1,
                1,
                R_e+300000., 
                1,
                0,
                0,
                0,
                0,
                3,
                4,
                2,
                10^-10,
                0.6,
                0,
                10, 
                600.,
                10^4,
                10^3)

Q_hist, orbit_hist, time_hist, mass_hist = Qlaw(orbit_inital, orbit_target, Q_Params, Sat_Params, time_initial)

print("Spent Prop: ", mass_hist[1] - mass_hist[end], "[kg]\n")
print("Transfer Time: ", time_hist[end] - time_hist[1], "[days]\n")

Spent Prop: 0.39397022723132125[kg]
Transfer Time: 0.6666666667442769[days]
Spent Prop: 0.2956234215741347[kg]
Transfer Time: 69.44444445252884[days]
